# TikTok Echo Chamber Model

## Background

The TikTok Echo Chamber model seeks to simulate the creation and sustenance of echo chambers on TikTok via bot to bot interactions. It utilizes agent-based modelling to demonstrate how agents - human or bot - of varying political leanings - conservative, progressive or neutral - interact with one another and take-on/lose each other's political leanings, eventually finding themselves surrounded by agents with similar leanings.

The model categorizes interactions like following, sharing, commenting and liking as positive interactions since they promote connection and increase the likelihood of political leanings being adopted. Conversely, interactions like unfollowing and disliking are thus categorized as negative interactions.

Agents are randomly placed on a graph. Human agents start off neutral to reflect a non-tailored For You Page (FYP) when a new user joins TikTok. There is at least one bot agent in the network and bot agents, as in real life, start off with a non-neutral political leaning with the goal of spreading that leaning. Human agents are able to have positive or negative interactions with other agents, while bot agents can only have positive interactions since they aim to spread their leanings. Upon a positive interaction, an agent tries to pass on its leaning to another. Upon a negative interaction, the receiving agent will try to change its political leaning to something other than the initiating agent's.

While individual human agents may interact with others of dissimilar leanings, the amplification of some leaning by bots near the human agent nonetheless leads to clusters of similar leanings.

## Implementation

This is a demonstration of running the TikTok Echo Chamber model in an IPython Notebook. The actual model and agent code are implemented in the src directory of this Github repo. Below, we will import the model class, instantiate it, run it, and plot the time series of the number of happy agents.

Please note that you may have to edit the source code since imports may not be identified in different settings. If the below import fails, make the following changes

In src/app.py change:
<code>from agents import</code> to <code>from src.agents import</code>
<code>from model import</code> to <code>from src.model import</code>

In src/model.py change:
<code>from agents import</code> to <code>from src.agents import</code>

## The following analyses are done in this notebook.
Notebook links don't work on Github so feel free to search by these titles.
1. [Effect of Number of Bots on Number of Clusters](#Effect-of-Number-of-Bots-on-Number-of-Clusters)
2. [Effect of Probability to Follow on Average Cluster Size](#Effect-of-Probability-to-Follow-on-Average-Cluster-Size)
3. [Comparing Bot-to-Bot Interactions to Echo Chamber Formation Rate](#Comparing-Bot-to-Bot-Interactions-to-Echo-Chamber-Formation-Rate)
4. [Comparing Bot-to-Human Ratio to Echo Chamber Formation Rate](#Comparing-Bot-to-Human-Ratio-to-Echo-Chamber-Formation-Rate)

[References](#references)

In [3]:
%pip install matplotlib pandas ipykernel


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

from src.model import TikTokEchoChamber

Now we instantiate a model instance: 10 nodes with 2 bot nodes and an average node degree of 3. The likelihood of agents to have a positive interaction with others is 40%. After a negative interaction, they have a 50% chance to become neutral.

In [5]:
ttec_model = TikTokEchoChamber(
    num_nodes=10,
    avg_node_degree=3,
    num_cons_bots=1,
    num_prog_bots=1,
    positive_chance=0.4,
    become_neutral_chance=0.5,
    seed=42
)

The model will run until all agents are of non-neutral leaning. However, there's no guarantee that a given model instantiation will *ever* settle down. So it will run for either 100 steps or until it stops on its own, whichever comes first:

In [ ]:
while ttec_model.running and ttec_model.steps < 100:
    ttec_model.step()
print(f"Number of Steps {ttec_model.steps}")  # Show how many steps have actually run

In [ ]:
from enum import IntEnum
import random
class POSInteraction(IntEnum):
    """Weight of each positive interaction"""
    FOLLOW = 5
    SHARE = 3
    LIKE = 2
    COMMENT = 2
    VIEW = 1

# define list of all positive/negative interaction weights
POSInteraction_LIST = list(map(int,POSInteraction))
class NEGInteraction(IntEnum):
    """Weight of each negative interaction"""
    UNFOLLOW = -5
    DISLIKE = -2


# define list of all positive/negative interaction weights
NEGInteraction_LIST = list(map(int, NEGInteraction))
random.choice(NEGInteraction_LIST)


Generate a pandas DataFrame of the number of conservative, progressive and neutral agent counts at the end of each step.

In [ ]:
model_out = ttec_model.datacollector.get_model_vars_dataframe()
model_out.head()

Now show the cluster metrics at the end of each step. This is in a specific table of the datacollector.

In [ ]:
cluster_out = ttec_model.datacollector.get_table_dataframe("CA")
cluster_out.head()

Finally, plot both series - political leanings and cluster analysis:

In [ ]:
model_out.plot()
cluster_out.plot()

***
# Effect of Number of Bots on Number of Clusters

How big do clusters get when agents are more likely to connect than disconnect?

Below we do a parameter sweep to see how the number of clusters changes with the number of bots.

First, we create a function which takes a model instance and returns the number of clusters ie. agents in  proximity to one another which also have similar leanings.

In [11]:
from batchrunner import batch_run

Now, we set up the batch run, with a dictionary of fixed and changing parameters. Let's keep everything fixed except for the Number of Bots. This will be deduced from the sum of Conservative and Progressive bots.

In [12]:
fixed_params = {"num_nodes": 30, "avg_node_degree": 3, "positive_chance": 0.5, "become_neutral_chance": 0.5}
variable_params = {"num_cons_bots": range(1, 9), "num_prog_bots": range(1, 9)}
all_params = fixed_params | variable_params

In [ ]:
results = batch_run(
    TikTokEchoChamber,
    parameters=all_params,
    iterations=10,
    max_steps=200,
)

# get dataframe of results
df = pd.DataFrame(results)
df

Plot the Effect of Number of Bots on Number of Clusters

In [ ]:
plt.scatter((df.num_cons_bots + df.num_prog_bots), df.CA_Num_Clusters)
plt.xlabel("Num of Bots")
plt.ylabel("Num of Clusters")
plt.grid()
plt.title("Effect of Number of Bots on Number of Clusters")
plt.show()

***
# Effect of Probability to Follow on Average Cluster Size

How big do clusters get when agents are more likely to connect than disconnect?

In [ ]:
from numpy import arange

fixed_params2 = {"num_nodes": 20, "avg_node_degree": 3, "num_cons_bots": 2, "num_prog_bots": 2, "become_neutral_chance": 0.5}
variable_params2 = {"positive_chance": arange(0.0, 1.0, 0.1)}
all_params2 = fixed_params2 | variable_params2

results2 = batch_run(
    TikTokEchoChamber,
    parameters=all_params2,
    iterations=10,
    max_steps=200,
)

df2 = pd.DataFrame(results2)
df2

In [ ]:
plt.scatter(df2.positive_chance, df2.CA_Avg_Cluster_Size)
plt.xlabel("Prob to Follow")
plt.ylabel("Avg Cluster Size")
plt.grid()
plt.title("Effect of Probability to Follow on Average Cluster Size")
plt.show()

***
# Comparing Bot-to-Bot Interactions to Echo Chamber Formation Rate

How do bot-to-bot interactions influence the number of steps taken to form echo chambers?

In [ ]:
fixed_params3 = {"num_nodes": 20, "avg_node_degree": 3, "num_cons_bots": 2, "num_prog_bots": 2, "positive_chance": 0.5, "become_neutral_chance": 0.5}

# TODO when #bot interactions is made, plot that instead
variable_params3 = {"num_cons_bots": range(1, 9), "num_prog_bots": range(1, 9)}
all_params3 = fixed_params3 | variable_params3

results3 = batch_run(
    TikTokEchoChamber,
    parameters=all_params3,
    iterations=10,
    max_steps=200,
)

df3 = pd.DataFrame(results3)
df3

In [ ]:
plt.scatter((df3.num_cons_bots + df3.num_prog_bots), df3.Step)
plt.xlabel("Num of Bots")
plt.ylabel("Num of Steps")
plt.grid()
plt.title("Effect of Number of Bots on Number of Steps")
plt.show()

***
# Comparing Bot-to-Human Ratio to Echo Chamber Formation Rate

How do bot-to-bot interactions influence the number of steps taken to form echo chambers?

In [ ]:
fixed_params4 = {"num_nodes": 20, "avg_node_degree": 3, "num_cons_bots": 2, "num_prog_bots": 2, "positive_chance": 0.5, "become_neutral_chance": 0.5}

variable_params4 = {"num_cons_bots": range(1, 9), "num_prog_bots": range(1, 9)}
all_params4 = fixed_params4 | variable_params4

results4 = batch_run(
    TikTokEchoChamber,
    parameters=all_params4,
    iterations=10,
    max_steps=200,
)

df4 = pd.DataFrame(results4)
df4

In [ ]:
no_bots = (df4.num_cons_bots + df4.num_prog_bots)
no_hums = df4.num_nodes - no_bots
bot_hum_ratio = no_bots.divide(no_hums, fill_value=1)

plt.scatter(bot_hum_ratio, df4.Step)
plt.xlabel("Bot to Human Ratio")
plt.ylabel("Num of Steps")
plt.grid()
plt.title("Bot to Human Ratio vs Number of Steps")
plt.show()

## References

This document was adopted from the Mesa Schelling model analysis file found at the mesa examples repository on GitHub
    [https://github.com/projectmesa/mesa/tree/main/mesa/examples/basic/schelling](https://github.com/projectmesa/mesa/tree/main/mesa/examples/basic/schelling)